In [40]:
import pandas as pd
from sqlalchemy import create_engine

In [41]:
# with open('MinimumWageData.csv', 'r') as f: # // open in read mode
#      text = f.read() # // read and save the text of file in a variable

# converted_text = text.replace("4..65(g,,j)", '')  #// replace characters, regex not required here, but this is where you will use it if required
# converted_text = text.replace("(g,j)", '')
# with open('MinimumWageData.csv', 'w') as f: # // open in write mode now
#       f.write(converted_text) # // write the file. Old data in the file will be removed

In [42]:
csv_file = "MinimumWageData.csv"
MinWage_data_df = pd.read_csv(csv_file, delimiter=',', quotechar='"')
MinWage_data_df.head()


,Year,State,Table_Data,Footnote,High.Value,Low.Value,CPI.Average,High.2018,Low.2018
0,1968,Alabama,...,NaN,0.00000,0.00000,34.783333,0.00,0.00
1,1968,Alaska,2.10,NaN,2.10000,2.10000,34.783333,15.12,15.12
2,1968,Arizona,18.72 - 26.40/wk(b),(b),0.66000,0.46800,34.783333,4.75,3.37
3,1968,Arkansas,1.25/day(b),(b),0.15625,0.15625,34.783333,1.12,1.12
4,1968,California,1.65(b),(b),1.65000,1.65000,34.783333,11.88,11.88


In [43]:
MinWage_data_df.dtypes

Year             int64
State           object
Table_Data      object
Footnote        object
High.Value     float64
Low.Value      float64
CPI.Average    float64
High.2018      float64
Low.2018       float64
dtype: object

In [45]:
csv_file = "output.csv"
Unemployment_data_df = pd.read_csv(csv_file)
Unemployment_data_df.head()

,Year,Month,State,County,Rate
0,2015,February,Mississippi,Newton County,6.1
1,2015,February,Mississippi,Panola County,9.4
2,2015,February,Mississippi,Monroe County,7.9
3,2015,February,Mississippi,Hinds County,6.1
4,2015,February,Mississippi,Kemper County,10.6


In [30]:
# Create a filtered dataframe from specific columns
MinWage_cols = ["State", "Year", "High.Value", "Low.Value", "CPI.Average"]
MinWage_transformed= MinWage_data_df[MinWage_cols].copy()

# Rename the column headers
MinWage_transformed = MinWage_transformed.rename(columns={"High.Value": "high_value",
                                                          "Low.Value": "low_value",
                                                          "CPI.Average": "cpi_average",
                                                          "State" : "state",
                                                          "Year":"year"
                                                          })

# Clean the data by and setting the index
#MinWage_transformed.set_index("state", inplace=True)

MinWage_transformed.head()

,state,year,high_value,low_value,cpi_average
0,Alabama,1968,0.00000,0.00000,34.783333
1,Alaska,1968,2.10000,2.10000,34.783333
2,Arizona,1968,0.66000,0.46800,34.783333
3,Arkansas,1968,0.15625,0.15625,34.783333
4,California,1968,1.65000,1.65000,34.783333


In [31]:
Unemployement_cols = ["State", "Year", "County", "Rate",]
Unemployement_transformed= Unemployment_data_df[Unemployement_cols].copy()

Unemployement_transformed = Unemployement_transformed.rename(columns={"County" : "county",
                                                                      "Rate" : "rate",
                                                                      "State" : "state",
                                                                      "Year":"year"
                                                                      })
Unemployement_transformed.head()

,state,year,county,rate
0,Mississippi,2015,Newton County,6.1
1,Mississippi,2015,Panola County,9.4
2,Mississippi,2015,Monroe County,7.9
3,Mississippi,2015,Hinds County,6.1
4,Mississippi,2015,Kemper County,10.6


In [32]:
#Match the range of years for the unemployement dataset. 
correct_year = ((MinWage_transformed['year'] > 1989) & (MinWage_transformed['year'] < 2017))



In [38]:
adjusted_MinWage_df = MinWage_transformed[correct_year]
adjusted_MinWage_df.head()

,state,year,high_value,low_value,cpi_average
1210,Alabama,1990,0.00,0.00,130.658333
1211,Alaska,1990,3.85,3.85,130.658333
1212,Arizona,1990,0.00,0.00,130.658333
1213,Arkansas,1990,3.25,3.25,130.658333
1214,California,1990,3.35,3.35,130.658333


In [34]:
#Transformation of the unemployment data to effectivley eliminate the county and month data with skewing the data. 
rollup_unemployement = Unemployement_transformed.groupby(['state', 'year']).mean().reset_index()
rollup_unemployement.head()

,state,year,rate
0,Alabama,1990,8.226990
1,Alabama,1991,9.081841
2,Alabama,1992,9.296891
3,Alabama,1993,9.182463
4,Alabama,1994,7.622139


In [35]:
rds_connection_string = "joefioro:Jfioro!23@localhost:5432/ETL_Project"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [36]:
engine.table_names()

OperationalError: (psycopg2.OperationalError) FATAL:  password authentication failed for user "joefioro"
 (Background on this error at: http://sqlalche.me/e/e3q8)

In [ ]:
Unemployement_transformed.to_sql(name='unemployment_rate', con=engine, if_exists='append', index=False)

In [ ]:
adjusted_MinWage_df.to_sql(name='minimum_wage', con=engine, if_exists='append', index=False)

In [ ]:
rollup_unemployement.to_sql(name='unemployment_by_state', con=engine, if_exists='replace', index=False)